In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Ejercicio 39

Calcule la correlación entre la cuota de mercado de una app y su sentimiento promedio. (⭐⭐)

Leo los csv usando únicamente las columnas que serán útiles para este ejercicio.

Cuota de mercado será el porcentaje de las descargas que tiene una aplicación sobre el total de descargas de todas las apps de su categoria. Esto es una aproximación asumiendo que cada persona tiene una única app de cada categoría (en la realidad no pasa, pero no tenemos otros datos para calcularla bien).

Sentimiento promedio de una app será el promedio de los valores que hay en la columna Sentiment_polarity, me quedo con esta columna y la del nombre de la aplicación en user_reviews. Usaré Translated_Review para eliminar reviews que se encuentran duplicadas en ese csv (están idénticas, las considero errores)


In [3]:
apps_details = pd.read_csv('/content/drive/MyDrive/Organización de Datos/googleplaystore.csv', usecols=['App', 'Category', 'Installs'])
user_reviews = pd.read_csv('/content/drive/MyDrive/Organización de Datos/googleplaystore_user_reviews.csv', usecols=['App', 'Translated_Review', 'Sentiment_Polarity'])

Hay apps que figuran repetidas. Estas no las tendré en cuenta ya que puedo tener valores que no coinciden en la cantidad de instalaciones, o en el caso de estar repetidos no aportan nada.



In [4]:
apps_details = apps_details.drop_duplicates(subset='App')

Además, hay apps que figuran con cantidad de instalaciones sin sentido, como "Free". Eliminaré apps que figuren así.

In [5]:
apps_details.Installs.unique() #'Free' no tiene sentido (no necesario resolución)

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+', '0', 'Free'], dtype=object)

In [6]:
apps_details = apps_details[apps_details.Installs != 'Free']

Para poder calcular la cuota de mercado de una aplicación, necesito que la columna Installs sea de tipo entero (es de tipo object), por lo que eliminaré el '+' de cada valor y me quedaré con la cifra redonda. Los cálculos serán una aproximación de la cantidad real de descargas, ya que, por ejemplo, no puedo adivinar cuantas instalaciones tuvo realmente una aplicación con '1+' instalaciones (en ese caso, asumiré que esa app tiene solo una descarga).

(También tengo que eliminar las comas para poder pasar a entero).

In [7]:
apps_details['Installs'] = apps_details['Installs'].map(lambda x: int(x.replace('+','').replace(',','')))

En el csv de user reviews, hay apps que tienen sentimientos con valor NaN, elimino estas ya que no aportan nada y no puedo inferir qué había en esas reviews. Además, elimino reviews duplicadas.

In [8]:
user_reviews[user_reviews.duplicated(subset=['App', 'Translated_Review'])].head(5) #ejemplo de reviews duplicadas (no necesario resolución)

,App,Translated_Review,Sentiment_Polarity
7,10 Best Foods for You,NaN,NaN
15,10 Best Foods for You,NaN,NaN
78,10 Best Foods for You,Good,0.7
79,10 Best Foods for You,Good,0.7
100,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,1.0


In [9]:
user_reviews[user_reviews.Sentiment_Polarity.isna()].head(5) #ejemplo de reviews con sentiment_polarity NaN (no necesario resolución)

,App,Translated_Review,Sentiment_Polarity
2,10 Best Foods for You,NaN,NaN
7,10 Best Foods for You,NaN,NaN
15,10 Best Foods for You,NaN,NaN
102,10 Best Foods for You,NaN,NaN
107,10 Best Foods for You,NaN,NaN


In [10]:
user_reviews = user_reviews.drop_duplicates(subset=['App', 'Translated_Review'])
user_reviews = user_reviews.dropna() #cualquier valor NaN en mis datos hace que no me sirva

In [11]:
apps_details["Category Installs"] = apps_details.groupby(["Category"]).transform("sum", numeric_only=True)

In [12]:
apps_details["Market Share"] = apps_details["Installs"] / apps_details["Category Installs"]

Ahora tengo la cuota de mercado para cada app.

Las columnas Category, Installs y Category Installs ya no me serán útiles, por lo que al hacer el merge no las tendré en cuenta.

Hago un inner join entre los dataframes ya que voy a necesitar datos de ambos para este ejercicio.

¿Por qué inner?

Si me quedo con una app que está en el primer df pero no en el segundo, no tendré el promedio sentiment_polarity (quedará NaN) para esta y no servirá.

Si tengo el caso opuesto, tendré el promedio sentiment_polarity para una app de la que desconozco su cuota de mercado, por lo que tampoco me servirá.

El merge lo hago entre el df que tiene apps y su market share, con un df que tiene apps y su sentimiento promedio. Se muestran ejemplos de apps en estos, no es necesario para la resolución.

In [13]:
apps_details[["App", "Market Share"]].head(3) #no necesario

,App,Market Share
0,Photo Editor & Candy Camera & Grid & ScrapBook,0.000087
1,Coloring book moana,0.004373
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",0.043730


In [14]:
user_reviews.groupby("App").mean(numeric_only=True).reset_index().head(3) #no necesario

,App,Sentiment_Polarity
0,10 Best Foods for You,0.465906
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.358227
2,11st,0.185943


In [15]:
apps_with_ms_and_polarity = pd.merge(apps_details[["App", "Market Share"]], user_reviews.groupby("App").mean(numeric_only=True).reset_index(), how='inner')

In [16]:
apps_with_ms_and_polarity.sample(5) #obtengo df con las app con market share y promedio de sentiment polarity (no necesario resolución)

,App,Market Share,Sentiment_Polarity
336,Barbie™ Fashion Closet,0.000721,0.043690
441,Hello Kitty Lunchbox,0.001129,-0.052718
565,Golfshot Plus: Golf GPS,0.000046,0.386179
130,Brilliant,0.002833,0.255497
224,Cookpad,0.047215,0.257437


Finalmente, la correlación entre la cuota de mercado de una app y su sentimiento promedio es -0.05 (casi 0, es decir, prácticamente nula correlación entre las variables).

In [17]:
apps_with_ms_and_polarity.corr(numeric_only=True).loc["Market Share", "Sentiment_Polarity"]

-0.05071648951225377